In [1]:
import pandas as pd
import numpy as np


In [2]:
filename = "NL2ML_ structure - levin.csv"

df = pd.read_csv(filename)

In [3]:
df.head(5)

,PK,Manually,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Automatically,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,chunk_id,notebook_id,splitting_id,code_block,data_format,graph_vertex,errors,graph_vertex_m1,graph_vertex_m2,graph_vertex_m3,...,python_methods_m3,python_methods_p1,python_methods_p2,python_methods_p3,kaggle_link,kaggle_comments,kaggle_upvotes,kaggle_section,kaggle_section_overview,kaggle_score
1,1,1,1,import os\nimport numpy as np\nimport matplotl...,None,Environment.import_modules,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,1,"print('Reading datasets...', end='')\n\ntrain_...",Table,Data: extraction.Load from CSV,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,1,"print('Preprocessing data...', end='')\n\nfeat...",Table,Data: transform.filter;Data: transform.data ty...,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,1,"print('Creating classifier...', end='')\nclf =...",None,Model: train.choose model class,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save the column names from the original markup file

In [4]:
HEADER = df.columns
HEADER = [col if ("Unnamed:" not in col) else "" for col in HEADER ]

In [5]:
df.columns = df.iloc[0,:]

In [6]:
df = df.drop(index=0)

In [7]:
df.head(5)

,chunk_id,notebook_id,splitting_id,code_block,data_format,graph_vertex,errors,graph_vertex_m1,graph_vertex_m2,graph_vertex_m3,...,python_methods_m3,python_methods_p1,python_methods_p2,python_methods_p3,kaggle_link,kaggle_comments,kaggle_upvotes,kaggle_section,kaggle_section_overview,kaggle_score
1,1,1,1,import os\nimport numpy as np\nimport matplotl...,None,Environment.import_modules,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,1,"print('Reading datasets...', end='')\n\ntrain_...",Table,Data: extraction.Load from CSV,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,1,"print('Preprocessing data...', end='')\n\nfeat...",Table,Data: transform.filter;Data: transform.data ty...,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,1,"print('Creating classifier...', end='')\nclf =...",None,Model: train.choose model class,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1,1,"print('Training classifier...', end='')\r\n%ti...",Table,Model: train.train model,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Functions for parsing libraries and comments

In [8]:
def get_libraries(text: str):
    text = str(text)
    libs = []
    line_array = [line.strip() for line in text.split("\n")]
    # delete all double spaces in all lines
    for ind in range(len(line_array)):
        
        prev_len = -1
        while prev_len != len(line_array[ind]):
            prev_len = len(line_array[ind])
            line_array[ind] = line_array[ind].replace("  ", " ")
    # now get libraries if line start with "import" or "from"
    for line in line_array:
        if line.startswith("import") or line.startswith("from"):
            libs.append(line.split(" ")[1].split(".")[0])
    return "\n".join(list(set(libs)))


def get_comments(text: str):
    text = str(text)
    comments = []
    line_array = [line.strip() for line in text.split("\n")]
    # now get libraries if line start with "import" or "from"
    for line in line_array:
        if line.startswith("#"):
            comments.append(line)
    return "\n".join(comments)

In [9]:
df['libraries'] = df.code_block.apply(get_libraries)

In [10]:
df['comments'] = df.code_block.apply(get_comments)

### Take the required column and move it to the desired step

In [11]:
buf_graph_vertex = list(df.graph_vertex.values)

### Index of the last not NaN cell (to cut from the bottom)

In [12]:
ind = len(buf_graph_vertex) - 1

for i in range(len(buf_graph_vertex) - 1, -1, -1):
    if str(buf_graph_vertex[i]) != "nan":
        break
    ind -= 1


In [13]:
ind

95

In [14]:
buf_graph_vertex = [np.NaN, np.NaN, np.NaN] + buf_graph_vertex

In [15]:
df.graph_vertex_m3 = buf_graph_vertex[:ind + 1] + [np.NaN, np.NaN, np.NaN] + buf_graph_vertex[ind + 4:-3]
buf_graph_vertex.pop(0)

df.graph_vertex_m2 = buf_graph_vertex[:ind + 1] + [np.NaN, np.NaN] + buf_graph_vertex[ind + 3:-2]
buf_graph_vertex.pop(0)

df.graph_vertex_m1 = buf_graph_vertex[:ind + 1] + [np.NaN] + buf_graph_vertex[ind + 2:-1]
buf_graph_vertex.pop(0)
buf_graph_vertex.pop(0)


df.graph_vertex_p1 = buf_graph_vertex + [np.NaN]
buf_graph_vertex.pop(0)

df.graph_vertex_p2 = buf_graph_vertex + [np.NaN, np.NaN]
buf_graph_vertex.pop(0)

df.graph_vertex_p3 = buf_graph_vertex + [np.NaN, np.NaN, np.NaN]
df.head(5)

,chunk_id,notebook_id,splitting_id,code_block,data_format,graph_vertex,errors,graph_vertex_m1,graph_vertex_m2,graph_vertex_m3,...,python_methods_m3,python_methods_p1,python_methods_p2,python_methods_p3,kaggle_link,kaggle_comments,kaggle_upvotes,kaggle_section,kaggle_section_overview,kaggle_score
1,1,1,1,import os\nimport numpy as np\nimport matplotl...,None,Environment.import_modules,FALSE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,1,"print('Reading datasets...', end='')\n\ntrain_...",Table,Data: extraction.Load from CSV,FALSE,Environment.import_modules,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,1,"print('Preprocessing data...', end='')\n\nfeat...",Table,Data: transform.filter;Data: transform.data ty...,FALSE,Data: extraction.Load from CSV,Environment.import_modules,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,1,"print('Creating classifier...', end='')\nclf =...",None,Model: train.choose model class,FALSE,Data: transform.filter;Data: transform.data ty...,Data: extraction.Load from CSV,Environment.import_modules,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1,1,"print('Training classifier...', end='')\r\n%ti...",Table,Model: train.train model,FALSE,Model: train.choose model class,Data: transform.filter;Data: transform.data ty...,Data: extraction.Load from CSV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Returning the original header markup

In [16]:
# add columns names 
df.loc[-1] = df.columns
# add original HEADER
df.columns = HEADER
df.index = df.index + 1
df.sort_index(inplace=True)

In [17]:
df.to_csv(filename[:-4] + "graph_vertex_shift.csv", index=False)